<a href="https://colab.research.google.com/github/SakshamTalwar12/medicalgen/blob/main/medicalfinetuningllm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (tokenizers)


In [13]:
!pip install huggingface_hub

In [14]:
pip install torch transformers datasets peft trl bitsandbytes

In [15]:
import torch
from trl import SFTTrainer
from peft import LoraConfig
from datasets import load_dataset
from transformers import (AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline)

In [16]:
llama_model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path = "aboonaji/llama2finetune-v2",
                                                   quantization_config = BitsAndBytesConfig(load_in_4bit = True,
                                                                                            bnb_4bit_compute_dtype = getattr(torch, "float16"),
                                                                                            bnb_4bit_quant_type = "nf4"),  device_map="auto")

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [17]:
llama_model.config.use_cache = False
llama_model.config.pretraining_tp = 1

In [18]:
llama_tokenizer= AutoTokenizer.from_pretrained(pretrained_model_name_or_path = "aboonaji/llama2finetune-v2", trust_remote_code=True)
llama_tokenizer.pad_token= llama_tokenizer.eos_token
llama_tokenizer.padding_side="right"

In [19]:
training_arguments= TrainingArguments(output_dir="./results", per_device_train_batch_size=1, max_steps=100)

In [20]:
llama_sft_trainer= SFTTrainer(model= llama_model, args=training_arguments, train_dataset =load_dataset(path= "aboonaji/wiki_medical_terms_llam2_format", split="train"), peft_config=LoraConfig ( task_type ="CAUSAL_LM", r= 8, lora_alpha=16, lora_dropout=0.1, target_modules=["q_proj", "v_proj"]))

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [21]:
llama_sft_trainer.train()

Step,Training Loss


TrainOutput(global_step=100, training_loss=1.7425677490234375, metrics={'train_runtime': 199.5903, 'train_samples_per_second': 0.501, 'train_steps_per_second': 0.501, 'total_flos': 2866181026775040.0, 'train_loss': 1.7425677490234375})

In [25]:
user_prompt="Please tell me about fever"
text_generation_pipeline=pipeline(task= "text-generation", model=llama_model, tokenizer=llama_tokenizer, max_length=300)
model_answer=text_generation_pipeline(f"<s>[INST] {user_prompt} [/INST]")
print(model_answer[0]['generated_text'])

Device set to use cuda:0


<s>[INST] Please tell me about fever [/INST]  Fever is a common medical condition characterized by an elevated body temperature, typically above 98. Unterscheidung between different types of fever, such as typhoid fever, malaria, and dengue fever, is important for proper diagnosis and treatment. Fever can be caused by a wide range of factors, including infections, autoimmune disorders, and cancer. Fever can be treated with medication, such as acetaminophen or ibuprofen, or through home remedies like drinking plenty of fluids and resting. However, it is important to seek medical attention if the fever is severe, lasts for an extended period of time, or is accompanied by other concerning symptoms.

Here are some key points to know about fever:

1. Definition: Fever is a temporary increase in the body's temperature, typically above 98°F (36.7°C).
2. Causes: Fever can be caused by a wide range of factors, including infections, autoimmune disorders, and cancer.
3. Types of fever: There are 

In [26]:
from IPython.display import clear_output

# This will clear all previous outputs in the notebook
clear_output(wait=True)
